# Import

In [1]:
import matplotlib.pyplot as plt
import pandas as pd
df=pd.read_csv('C:\\Users\\Armin\\Documents\\Code\\Studienarbeit\\crypto_daten_kaggle\\train.csv')
df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,2,40.0,2376.5800,2399.5000,2357.1400,2374.5900,19.233005,2373.116392,-0.004218
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.380000,8.530000,-0.014399
2,1514764860,1,229.0,13835.1940,14013.8000,13666.1100,13850.1760,31.550062,13827.062093,-0.014643
3,1514764860,5,32.0,7.6596,7.6596,7.6567,7.6576,6626.713370,7.657713,-0.013922
4,1514764860,7,5.0,25.9200,25.9200,25.8740,25.8770,121.087310,25.891363,-0.008264


In [3]:
details=pd.read_csv('C:\\Users\\Armin\\Documents\\Code\\Studienarbeit\\crypto_daten_kaggle\\asset_details.csv')
details.head()

,Asset_ID,Weight,Asset_Name
0,2,2.397895,Bitcoin Cash
1,0,4.304065,Binance Coin
2,1,6.779922,Bitcoin
3,5,1.386294,EOS.IO
4,7,2.079442,Ethereum Classic


# Percentage of Nan values

In [16]:
import numpy as np

for i in range(14):
    
    asset_id = details[details["Asset_ID"] == i]["Asset_Name"]
    dfcrop=df[df['Asset_ID']==i]
    print( asset_id.iloc[-1] +' percentage of values not nan',(1-(np.sum((dfcrop['Target'].isnull()).astype(int))/dfcrop.shape[0]))*100)
    


Binance Coin percentage of values not nan 99.3094374141301
Bitcoin percentage of values not nan 99.9844603180932
Bitcoin Cash percentage of values not nan 99.75116928934543
Cardano percentage of values not nan 98.95466572016784
Dogecoin percentage of values not nan 87.46838441098623
EOS.IO percentage of values not nan 99.88225907096167
Ethereum percentage of values not nan 99.9826193640732
Ethereum Classic percentage of values not nan 99.52201983776557
IOTA percentage of values not nan 87.87076707006158
Litecoin percentage of values not nan 99.97336441670117
Maker percentage of values not nan 94.03054748939965
Monero percentage of values not nan 85.91615278314144
Stellar percentage of values not nan 96.52474857329506
TRON percentage of values not nan 98.87899026971662


# Recreate Targets

In [19]:
from tqdm import tqdm
def compute(df):
    
    R=list()
    c=list(df['Close'])
    for i in range(df.shape[0]):
        future=c[min([i+16,df.shape[0]-1])]
        past=c[min([i+1,df.shape[0]-1])]
        R.append(future/past)
    R=np.array(R)

    df['pred']=R-1
    return df
crops=list()
for a in tqdm(list(range(14))):
    
    crops.append(compute(df[df['Asset_ID']==a]))


  0%|          | 0/14 [00:00<?, ?it/s]C:\Users\Armin\AppData\Local\Temp/ipykernel_10404/2698420633.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['pred']=R-1
100%|██████████| 14/14 [00:29<00:00,  2.13s/it]


In [20]:

conc=pd.concat(crops)

In [21]:
conc.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target,pred
1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399,-0.014399
9,1514764920,0,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875,-0.015875
17,1514764980,0,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410,-0.015410
25,1514765040,0,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524,-0.012524
33,1514765100,0,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940,-0.005940


In [22]:
conc=conc.reset_index()

In [23]:
j=np.array(list(conc['Target'].isnull()))
new_targets=np.where(j,conc['pred'],conc['Target'])

In [24]:
conc['Target']=new_targets
conc=conc.drop(columns=['pred'],axis=1)
conc.head()

,index,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399
1,9,1514764920,0,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875
2,17,1514764980,0,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410
3,25,1514765040,0,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524
4,33,1514765100,0,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940


# No more nan values 

In [25]:
import numpy as np

for i in range(14):
    dfcrop=conc[conc['Asset_ID']==i]
    print('Percentage of values not nan',(1-(np.sum((dfcrop['Target'].isnull()).astype(int))/dfcrop.shape[0]))*100)
  
    


Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0
Percentage of values not nan 100.0


# Save new train.h5 file h5 is way faster

In [26]:
conc.to_hdf('train.h5',key='conc', mode='w',index=False)

In [27]:
del conc,crops

# Load your saved h5 file

In [28]:
df=pd.read_hdf('train.h5')
df=df.drop(columns=['index'])
df.head()

,timestamp,Asset_ID,Count,Open,High,Low,Close,Volume,VWAP,Target
0,1514764860,0,5.0,8.5300,8.5300,8.5300,8.5300,78.38,8.530000,-0.014399
1,1514764920,0,7.0,8.5300,8.5300,8.5145,8.5145,71.39,8.520215,-0.015875
2,1514764980,0,45.0,8.5065,8.5299,8.4848,8.4848,1546.82,8.501394,-0.015410
3,1514765040,0,14.0,8.5009,8.5066,8.4744,8.5009,125.80,8.479810,-0.012524
4,1514765100,0,5.0,8.5007,8.5007,8.4560,8.4560,125.01,8.458435,-0.005940


In [29]:
df.shape

(24236806, 10)